In [19]:
import os
import requests
from dotenv import load_dotenv
from openai import OpenAI
import json
import gradio as gr
from duckduckgo_search import DDGS
from pydantic import BaseModel, Field
from langchain_core.tools import tool
from langchain_core.utils.function_calling import convert_to_openai_tool

In [2]:
load_dotenv(override=True)

openai_api = os.getenv("OPENAI_API_KEY")
serper_api = os.getenv("X-API-KEY")

if openai_api:
    print(f"Exists and starts with {openai_api[:10]}")

if serper_api:
    print(f"Exists and starts with {serper_api[:10]}")

Exists and starts with sk-proj-61
Exists and starts with db85bd4af6


In [3]:
openai = OpenAI()
MODEL = "gpt-5-mini"

In [4]:
system_message = """
You are a helpful assistant for an Airline called FlightAI.
Give short, courteous answers, no more than 1 sentence.
Always be accurate. If you don't know the answer, say so.
"""

In [5]:
ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(city):
    print(f"Tool called for {city.lower()}")
    price = ticket_prices.get(city.lower(), "Unknown city")
    return f"The price for {city.lower()} is {price}"

In [6]:
get_ticket_price('PARIS')

Tool called for paris


'The price for paris is $899'

In [7]:
price_function = {
    "type": "function",
    "function": {
        "name": "get_ticket_price",
        "description": "Returns the cost of a round-trip flight to a specific city.",
        "parameters": {
            "type": "object",
            "properties": {
                "destination_city": {
                    "type": "string",
                    "description": "The city name, e.g., 'London'"
                },
            },
            "required": ["destination_city"],
            "additionalProperties": False
        }
    }
}

In [8]:
def search_internet(query):
    print(f"Tool called for '{query}'")
    url = "https://google.serper.dev/search"
    payload = json.dumps({"q": query, "gl": "us", "hl": "en"})
    headers = {
        'X-API-KEY': serper_api,
        'Content-Type': 'application/json'
    }

    response = requests.post(url, headers=headers, data=payload)
    results = response.json().get('organic', [])
    
    snippets = [r.get('snippet', '') for r in results[:3]]
    return "\n\n".join(snippets) if snippets else "No results found."

In [9]:
search_internet("what is the weather like in Tirana?")

Tool called for 'what is the weather like in Tirana?'


'Overcast skies with heavy rain likely late. Low 49F. Winds SSE at 10 to 20 mph. Chance of rain 100%. Rainfall possibly over one inch. Localized flooding is ...\n\nA clear sky and light windsSunny intervals and a gentle breezeThundery showers and light windsSunny and light windsSunny and light windsLight rain and light ...\n\nTirana 7 day weather forecast including weather warnings, temperature, rain ... 6°C Minimum feels like temperature: 6° Celsius;. Feels like temperature.'

In [10]:
search_tool = {
    "type": "function",
    "function": {
        "name": "search_internet",
        "description": "Search the web for current events or facts.",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string", 
                    "description": "The search query"
                    },
            },
            "required": ["query"],
            "additionalProperties": False
        }
    }
}

In [11]:
def send_email(recipient, subject, body):
    print(f"Tool called for sending an email")
    # this is where the SMTP logic will be
    print(f"--- EMAIL SENT ---")
    print(f"To: {recipient}")
    print(f"Subject: {subject}")
    print(f"Body: {body}")
    print(f"------------------")
    return f"Successfully sent email to {recipient}"

In [12]:
class EmailArgs(BaseModel):
    recipient: str = Field(
        description="The destination email address, e.g., 'example@mail.com'"
    )
    subject: str = Field(
        description="A short, descriptive subject line for the email."
    )
    body: str = Field(
        description="The full text content of the email message."
    )

email_tool = {
    "type": "function",
    "function": {
        "name": "send_email",
        "description": "Sends an email message to a specified recipient.",
        "parameters": EmailArgs.model_json_schema()
    }
}

In [26]:
@tool
def get_weather(city: str):
    """
    Get the weather for a city using the free Open-Meteo API. 
    Use this to check current temperature and conditions.
    """

    print(f"Calling Weather Tool for {city}")

    geo_url = f"https://geocoding-api.open-meteo.com/v1/search?name={city}&count=1"
    geo_res = requests.get(geo_url).json()
    
    if not geo_res.get('results'):
        return f"Could not find coordinates for {city}."
    
    lat = geo_res['results'][0]['latitude']
    lon = geo_res['results'][0]['longitude']

    weather_url = f"https://api.open-meteo.com/v1/forecast?latitude={lat}&longitude={lon}&current_weather=true"
    res = requests.get(weather_url).json()
    
    current = res['current_weather']
    return f"The current weather in {city} is {current['temperature']}°C with a windspeed of {current['windspeed']} km/h."

In [27]:
get_weather.invoke("Tirana")

Calling Weather Tool for Tirana


'The current weather in Tirana is 13.5°C with a windspeed of 7.0 km/h.'

In [20]:
get_weather_json = convert_to_openai_tool(get_weather)
get_weather_json

{'type': 'function',
 'function': {'name': 'get_weather',
  'description': 'Get the weather for a city using the free Open-Meteo API. \nUse this to check current temperature and conditions.',
  'parameters': {'properties': {'city': {'type': 'string'}},
   'required': ['city'],
   'type': 'object'}}}

In [21]:
tools = [price_function, search_tool, email_tool, get_weather_json]
tools

[{'type': 'function',
  'function': {'name': 'get_ticket_price',
   'description': 'Returns the cost of a round-trip flight to a specific city.',
   'parameters': {'type': 'object',
    'properties': {'destination_city': {'type': 'string',
      'description': "The city name, e.g., 'London'"}},
    'required': ['destination_city'],
    'additionalProperties': False}}},
 {'type': 'function',
  'function': {'name': 'search_internet',
   'description': 'Search the web for current events or facts.',
   'parameters': {'type': 'object',
    'properties': {'query': {'type': 'string',
      'description': 'The search query'}},
    'required': ['query'],
    'additionalProperties': False}}},
 {'type': 'function',
  'function': {'name': 'send_email',
   'description': 'Sends an email message to a specified recipient.',
   'parameters': {'properties': {'recipient': {'description': "The destination email address, e.g., 'example@mail.com'",
      'title': 'Recipient',
      'type': 'string'},
     

In [28]:
def handle_tool_calls(message):
    responses = []
    for tool_call in message.tool_calls:
        function_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        
        if function_name == "get_ticket_price":
            city = arguments.get("destination_city")
            result = get_ticket_price(city)
        elif function_name == "search_internet":
            query = arguments.get("query")
            result = search_internet(query)
        elif function_name == "send_email":
            raw_args = json.loads(tool_call.function.arguments)
            args = EmailArgs(**raw_args)
            result = send_email(
                recipient=args.recipient, 
                subject=args.subject, 
                body=args.body
            )
        elif function_name == "get_weather":
            result = get_weather.invoke(arguments)
        else:
            result = "Error: Tool not found."

        responses.append({
            "role": "tool",
            "content": str(result),
            "tool_call_id": tool_call.id
        })
        
    return responses

In [29]:
def chat(message, history):
    history = [{"role":h[ "role"], "content":h ["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    while response.choices[0].finish_reason == "tool_calls":
        message = response.choices[0].message
        responses = handle_tool_calls(message)
        messages.append(message)
        messages.extend(responses)
        response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    return response.choices[0].message.content

In [30]:
gr.ChatInterface(fn=chat, type="messages", flagging_mode="never").launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


Tool called for paris
Tool called for tokyo
Tool called for tirana
Tool called for tirana
Tool called for 'Vienna to Tirana flight price 22 February 2026'
Calling Weather Tool for Tirana
Tool called for sending an email
--- EMAIL SENT ---
To: hutavirvi07@gmail.com
Subject: Flight & Weather Summary — Vienna to Tirana (22 Feb 2026)
Body: Hi Virvi,

Here’s a quick summary of what we discussed:

- Paris round-trip price: $899.
- Tokyo round-trip price: $1,400.
- Vienna → Tirana (22 Feb 2026) fares: starting at about $40 round-trip (one-way from about $23).
- Current Tirana weather: 13.5°C with 7.0 km/h wind — consider packing a light jacket.

Safe travels,
FlightAI
------------------
